In [ ]:
# Kütüphanelerin Yüklenmesi
!pip install keras

In [2]:
import numpy
import sys
# Keras and LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [4]:
# Txt Dosyasının Açılması
myfile = open("nirvananota.txt","r")
data = myfile.read().split("\n")

In [5]:
# Boşlukların temizlenmesi bütün notaların tek tek eleman olarak alınması
clean = []
for i in range(0,len(data)):
    clean = clean + [x for x in data[i].split(" ") if str(x) != '']
    
    
clean[:10]

['Em', 'G', 'Em', 'G', 'Em', 'G', 'Em', 'G', 'Em', 'G']

In [6]:
# Karakterler için Dictionary Oluşturulması
chars = sorted(list(set(clean)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
char_to_int

{'A': 0,
 'A#': 1,
 'A#7': 2,
 'A7': 3,
 'Ab': 4,
 'Am': 5,
 'B': 6,
 'B7': 7,
 'Bm7': 8,
 'C': 9,
 'C#': 10,
 'C#7': 11,
 'C#m': 12,
 'C7': 13,
 'D': 14,
 'D#7': 15,
 'D7': 16,
 'Dm': 17,
 'Dm#': 18,
 'E': 19,
 'E7': 20,
 'Em': 21,
 'EmC': 22,
 'F': 23,
 'F#': 24,
 'F#7': 25,
 'F#m': 26,
 'F7': 27,
 'G': 28,
 'G#': 29,
 'G#7': 30,
 'G#m': 31,
 'G#m,': 32,
 'G7': 33,
 'new': 34,
 'sleep': 35}

In [8]:
# Toplam Nota ve Karakter Sayımız
n_chars = len(clean)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  2569
Total Vocab:  36


In [9]:
# Notaların birbirlerine bağlanması örneğin ;
# E7 G E C --> E7 G E TARGET/C
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = clean[i:i + seq_length]
	seq_out = clean[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  2559


# Veriyi Modele Vermek İçin Hazırlıyoruz

In [10]:
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize Etmek.
X = X / float(n_vocab)
# Y değişkeni için One Hot Encoding Yapıyoruz.
y = np_utils.to_categorical(dataY)

### Model Yapısı 3 Layerli Stacked LSTM.

In [11]:
model = Sequential()
model.add(LSTM(512, return_sequences=True,
                input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Bu kısımda modelin ağırlıklarını kaydetmek için şu şekilde bir metod uyguluyoruz ;

- Eğer yeni epochta şu ana kadarki en düşük loss varsa onun ağırlıklarını kaydet yoksa devam et.

In [12]:
filepath="stacked-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=100, batch_size=5, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
2559/2559 [==============================] - 84s 33ms/step - loss: 3.2784

Epoch 00001: loss improved from inf to 3.27843, saving model to stacked-weights-improvement-01-3.2784.hdf5
Epoch 2/100
2559/2559 [==============================] - 79s 31ms/step - loss: 3.2096

Epoch 00002: loss improved from 3.27843 to 3.20958, saving model to stacked-weights-improvement-02-3.2096.hdf5
Epoch 3/100
2559/2559 [==============================] - 78s 30ms/step - loss: 3.2259

Epoch 00003: loss did not improve from 3.20958
Epoch 4/100
2559/2559 [==============================] - 79s 31ms/step - loss: 3.1036

Epoch 00004: loss improved from 3.20958 to 3.10363, saving model to stacked-weights-improvement-04-3.1036.hdf5
Epoch 5/100
2559/2559 [==============================] - 78s 30ms/step - loss: 2.9297

Epoch 00005: loss improved from 3.10363 to 2.92973, saving model to stacked-weights-improvement-05-2.9297.hdf5
Epoch 6/100
2559/2559 [=======

# Modelin en düşük loss'lu ağırlıklı halini çağırıyoruz.

In [14]:
filename = "stacked-weights-improvement-90-0.2360.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

### Bu kısımda olay başlıyor. Yapılan olay şu, öncelikle başlangıç değerleri veriliyor bunlara göre predictionlar yapılarak sürekli yeni nota tahminleniyor ardından sürekli başlangıç değerleri silinerek devam ediliyor. 

### Örnek
- D B7 D B7 A G F# sleep A F için bir nota tahmin edildi. Mesela buna **B7** diyelim. Ardından
- B7 D B7 A G F# sleep A F B7 şeklinde verilip yeni tahmin alınıyor.

In [28]:
deneme = []
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(150):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result+str(" "))
    pattern.append(index)
    deneme.append(index)
    pattern = pattern[1:len(pattern)]
deneme = deneme[len(pattern):]
print("\nDone.")

Seed:
" E7E7EF#GEAGBB "
F# E7 sleep E7 A G B B F# E7 E7 A G B B F# E7 new E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G B7 G E sleep E E A G 
Done.
